<a href="https://colab.research.google.com/github/MarcinLass/Point-Cloud-Registration/blob/master/SolidCentroidDXF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%pip install ezdxf

import ezdxf


def calculate_centroid(vertices):
    """
    Calculate the centroid of a set of vertices.
    """
    if len(vertices) == 0:
        return None  # No vertices to process

    # Initialize sums for x, y, z
    x_sum, y_sum, z_sum = 0, 0, 0

    # Sum up coordinates
    n = 0

    for vertex in vertices:
        if vertex[0] != 0: #checking if the vertices is not 0.0 0.0 0.0
            x_sum += vertex[0]
            y_sum += vertex[1]
            z_sum += vertex[2]

            n+=1 #counting number of "real" vertices


    # Calculate centroid

    return (x_sum / n, y_sum / n, z_sum / n)


def extract_polylines_and_calculate_centroids(dxf_file, output_csv):
    """
    Extract all POLYLINE entities from a DXF file, calculate their centroids, and save to a CSV file.
    """
    import csv

    # Read the DXF file
    doc = ezdxf.readfile(dxf_file)
    msp = doc.modelspace()

    # Prepare to write CSV
    with open(output_csv, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        # writer.writerow(["ID", "Centroid_X", "Centroid_Y", "Centroid_Z"])  # CSV header

        polyline_id = 1  # Unique ID for each polyline
        for entity in msp.query("POLYLINE"):
            try:
                # Extract vertices
                vertices = []
                for vertex in entity.vertices:
                    vertices.append(tuple(vertex.dxf.location))
                    #print(vertex.dxf.location[0], vertex.dxf.location[1], vertex.dxf.location[2])
                print (vertices)

                # Calculate centroid
                centroid = calculate_centroid(vertices)

                if centroid:
                    # Write to CSV
                    writer.writerow(["Part1_" + str(polyline_id), centroid[0],centroid[1],centroid[2]])
                    print(f"Polyline ID {polyline_id}: Centroid = {centroid}")
                else:
                    print(f"Polyline ID {polyline_id} has no vertices.")

                polyline_id += 1
            except Exception as e:
                print(f"Error processing polyline ID {polyline_id}: {e}")
                polyline_id += 1



dxf_file = "test2.dxf"  # Path to your DXF file
output_csv = "centroids.txt"  # Path to output CSV file

extract_polylines_and_calculate_centroids(dxf_file, output_csv)

[(5277.139048, 4894.072125, 115.63415), (5276.829048, 4893.845595, 115.63415), (5276.829048, 4893.845595, 115.19415), (5277.139048, 4894.072125, 115.19415), (5276.598947, 4894.160481, 115.19415), (5276.908947, 4894.387011, 115.19415), (5276.598947, 4894.160481, 115.63415), (5276.908947, 4894.387011, 115.63415), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0)]
Polyline ID 1: Centroid = (5276.8689975, 4894.116303, 115.41414999999999)
[(5243.004665, 4889.781504, 109.774533), (5243.004665, 4889.354935, 109.774533), (5243.004665, 4889.781504, 108.229991), (5243.004665, 4889.834935, 108.229991), (5243.004665, 4889.834935, 107.877266), (5243.004665, 4889.354935, 107.877266), (5243.162269, 4889.834935, 108.229991), (5243.162269, 4889.781504, 108.229991), (5243.162269, 4889.781504, 109.774533), (5243.162269, 4889.834935, 109.774533), (5243.38

In [39]:
import csv
import numpy as np

def read_csv(file_path):
    data = []
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        # next(reader)  # Skip header if there is one
        for row in reader:
            id_val = row[0]  # ID
            coords = list(map(float, row[1:]))  # Assuming next 3 columns are XYZ coordinates
            data.append((id_val, coords))
    return data


def write_csv(output_file, result_data):
    with open(output_file, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID1', 'X1', 'Y1', 'Z1', 'ID2'])  # Write header
        writer.writerows(result_data)

def euclidean_distance(coord1, coord2):
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2 + (coord1[2] - coord2[2])**2)



def main(file1, file2, output_file, N=2):
    # Step 1: Read the data from both CSVs
    data1 = read_csv(file1)  # Data from the first CSV
    data2 = read_csv(file2)  # Data from the second CSV

    result_data = []

    counter = 0

    # Step 2: Iterate through each row in the first CSV
    for id1, coords1 in data1:
        distances = []
        print("Obliczanie dla: " + id1)

        # Step 3: For each row in file1, calculate the distance to each row in file2
        for id2, coords2 in data2:
            dist = euclidean_distance(coords1, coords2)
            counter += 1
            distances.append((dist, id2))

        # Step 4: Sort the distances and get the N smallest
        distances.sort(key=lambda x: x[0])  # Sort by the first element (distance)

        # Step 5: Add the N closest rows to the result_data
        for i in range(N):
            dist, id2 = distances[i]
            result_data.append((id1, *coords1, id2))  # Add ID1, XYZ coordinates, and ID2

    # Step 6: Write the results to a new CSV
    write_csv(output_file, result_data)
    print(counter)




file1 = "centroids.txt"
file2 = "test2-FINAL-SCANS-COORDINATES.txt"

output_file = "result.txt"

main(file1,file2,output_file,15)


#c = read_csv(centroids_file)


# print(type(c))
# print(c[0][0])
# print(c[0][1][0])
# a = c[0][1][0] + c[0][1][1]
# print(type(c[0][1][0]))
# print(a)

Obliczanie dla: Part1_1
Obliczanie dla: Part1_2
Obliczanie dla: Part1_3
Obliczanie dla: Part1_4
Obliczanie dla: Part1_5
Obliczanie dla: Part1_6
Obliczanie dla: Part1_7
Obliczanie dla: Part1_8
Obliczanie dla: Part1_9
Obliczanie dla: Part1_10
Obliczanie dla: Part1_11
Obliczanie dla: Part1_12
Obliczanie dla: Part1_13
Obliczanie dla: Part1_14
Obliczanie dla: Part1_15
Obliczanie dla: Part1_16
Obliczanie dla: Part1_17
Obliczanie dla: Part1_18
Obliczanie dla: Part1_19
Obliczanie dla: Part1_20
Obliczanie dla: Part1_21
Obliczanie dla: Part1_22
Obliczanie dla: Part1_23
Obliczanie dla: Part1_24
Obliczanie dla: Part1_25
Obliczanie dla: Part1_26
Obliczanie dla: Part1_27
Obliczanie dla: Part1_28
Obliczanie dla: Part1_29
Obliczanie dla: Part1_30
Obliczanie dla: Part1_31
Obliczanie dla: Part1_32
Obliczanie dla: Part1_33
Obliczanie dla: Part1_34
Obliczanie dla: Part1_35
Obliczanie dla: Part1_36
Obliczanie dla: Part1_37
Obliczanie dla: Part1_38
Obliczanie dla: Part1_39
Obliczanie dla: Part1_40
Obliczani